In [1]:
import requests
import json
import datetime
import base64
from urllib.parse import urlencode

In [2]:
class SpotifyClient:
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
        self.token_url = 'https://accounts.spotify.com/api/token'
        self.token_method = 'POST'

        self.access_token = None
        self.token_expires = datetime.datetime.now()
        self.access_token_did_expire = True 
    

    def get_token_data(self):
        token_data = {
            'grant_type': 'client_credentials'
        }
        return token_data

    def get_client_credentials(self, decode = False):
        if self.client_secret == None or self.client_secret == None:
            raise Exception("You must include your client id and its secret keys")

        client_creds = f'{self.client_id}:{self.client_secret}'
        client_creds_base64 = base64.b64encode(client_creds.encode())

        if decode:
            client_creds_base64_decode = client_creds_base64.decode()
            return client_creds_base64_decode
        
        else:
            return client_creds_base64
    
    def get_token_header(self):
        client_credentials = self.get_client_credentials(decode=True)
        token_headers = {
            'Authorization': f'Basic {client_credentials}'
        }
        return token_headers
        
    
    def perform_auth(self, show_auth = False):
        user_token_data = self.get_token_data()
        user_token_headers = self.get_token_header()

        r = requests.post(url=self.token_url, data=user_token_data, headers=user_token_headers)
        if not r.status_code in range(200, 299):
            raise Exception('Could not authenticate client')
            return False 
            
        if show_auth:
            print(json.dumps(r.json()))
            
        token_response_data = r.json()
        now = datetime.datetime.now()

        self.access_token = token_response_data['access_token']
        access_token_expires_in = token_response_data['expires_in']
        self.token_expires = now + datetime.timedelta(seconds=access_token_expires_in)
        self.access_token_did_expire = self.token_expires < now 

        return True 
    
    def get_access_token(self):
        """
        This function checks whether the authentication is done 
        successfully or not. In case, the authentication validity
        expires, then it tries to re-authenticate with the help of 
        ``` perform_auth() ``` function
        """
        token = self.access_token
        expires = self.token_expires
        now = datetime.datetime.now()
        if now > expires:
            self.perform_auth()
            return self.get_access_token() # recursive call everytime if validity fails
        
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        
        print('OK')
        return token
        
    def get_resource_headers(self):
        access_token = self.get_access_token()
        endpoints = 'https://api.spotify.com/v1/search'
        headers = {
            'Authorization': f'Bearer {access_token}'
        }
        return headers
    
    
    def get_resource(self, lookup_id, resource_type='albums', version = 'v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_headers()
        r = requests.get(endpoint, headers=headers)
        print(r.status_code)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    '''
    (function under developement)
    
    def get_artist(self, get_most_popular=False, id=None, query=None):
        if not get_most_popular and id != None:
            return self.get_resource(id, resource_type='artists', version='v1')
        elif get_most_popular and query != None:
            search = self.search(query=query)
            artist_id = search['tracks']['items'][0]['album']['artists'][0]['id']
            return artist_id
        else:
            raise Exception('Either give the query or specify the id')
    '''
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists', version='v1')
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums', version='v1')
    
    def get_playlist(self, _id):
        return self.get_resource(_id, resource_type='playlists', version='v1')
    
    
    def search(self, query, query_type='track'):
        endpoints = 'https://api.spotify.com/v1/search'
        headers = self.get_resource_headers()
        
        data = urlencode({
            'q' : query,
            'type': query_type
        })
        
        lookup_url = f'{endpoints}?{data}'
        r = requests.get(lookup_url, headers = headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

In [3]:
client_id = '2476c1f2ab024aae8debc9146cd01926'
client_secret = '5628b40e45294570a9b115af35654b64'

In [4]:
sc = SpotifyClient(client_id, client_secret)
sc.perform_auth(show_auth=True)

{"access_token": "BQCcrOmn6mhZNkQc8lFTfiqEzZ_cWsm8xWoB7gR-BkEd6qDjM8Hhy2T92Z0m8hKdm65dK9zpRvpOS4BGDMg", "token_type": "Bearer", "expires_in": 3600}


True

### Using the search method

In [5]:
sc.search('Kontrol')

OK


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Kontrol&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/520qA5VGL9iI0SbmEnTVNg'},
       'href': 'https://api.spotify.com/v1/artists/520qA5VGL9iI0SbmEnTVNg',
       'id': '520qA5VGL9iI0SbmEnTVNg',
       'name': 'Maleek Berry',
       'type': 'artist',
       'uri': 'spotify:artist:520qA5VGL9iI0SbmEnTVNg'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BZ',
      'CA',
      'CH',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
      'DM',
      'DO',
      'DZ',
      'EC',
      

In [12]:
artist = sc.get_artist('520qA5VGL9iI0SbmEnTVNg')
artist

OK
200


{'external_urls': {'spotify': 'https://open.spotify.com/artist/520qA5VGL9iI0SbmEnTVNg'},
 'followers': {'href': None, 'total': 236687},
 'genres': ['afropop',
  'azontobeats',
  'dancehall',
  'nigerian pop',
  'swedish dancehall',
  'uk dancehall'],
 'href': 'https://api.spotify.com/v1/artists/520qA5VGL9iI0SbmEnTVNg',
 'id': '520qA5VGL9iI0SbmEnTVNg',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/bb6f41d155634330c17b4ea9a4fe34eb23d3fd8b',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/1fb44e4197100484d3aa4fe05ae8ed1e74050a40',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/0f78b65272644bd3ca7e4ca8e0f04f83679bce25',
   'width': 160}],
 'name': 'Maleek Berry',
 'popularity': 63,
 'type': 'artist',
 'uri': 'spotify:artist:520qA5VGL9iI0SbmEnTVNg'}

In [7]:
search = sc.search('Kontrol')

OK


In [8]:
search['tracks']['items'][0]['album']['artists'][0]['id']

'520qA5VGL9iI0SbmEnTVNg'

In [13]:
album = sc.get_album('0qvdWQTD4DeilItZUvHecT')
album

OK
200


{'album_type': 'single',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/520qA5VGL9iI0SbmEnTVNg'},
   'href': 'https://api.spotify.com/v1/artists/520qA5VGL9iI0SbmEnTVNg',
   'id': '520qA5VGL9iI0SbmEnTVNg',
   'name': 'Maleek Berry',
   'type': 'artist',
   'uri': 'spotify:artist:520qA5VGL9iI0SbmEnTVNg'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BZ',
  'CA',
  'CH',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',
  'KG',
  'KH',
  'KI',
  'KM',
  'KN',
  'KR',
  'K